Summary

The provided code script reads blog text data, applies the TextRank algorithm for text summarization with different compression rates, and generates summaries for each blog. It then evaluates the generated summaries using Rouge metrics by comparing them to reference summaries. The script saves the generated summaries as a new CSV file and computes the minimum, mean, and maximum Rouge scores, providing an analysis of the summarization quality. Overall, the script performs text summarization and evaluates the effectiveness of the approach using Rouge metrics.

### Libraries

In [2]:
import pandas as pd
from rouge import Rouge
import pandas as pd
import random 
import os
from nltk.tokenize import sent_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
!pip install en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
!pip install datasets
import os
import re
from datasets import load_dataset
import pandas as pd
import requests
import json
from statistics import mean
import random
import csv
import nltk
from nltk.corpus import gutenberg
import random
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda-autodetect,transformers,lookups]'
!python -m spacy download en_core_web_sm
!pip install en_core_web_sm
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize


  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/12/17/95e48481a826f85c918e0610257de493164096e29fe16ec408c0b862e057/datasets-2.13.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/e7/41/96ac938770ba6e7d5ae1d8c9cafebac54b413549042c6260f0d0a6ec6622/multiprocess-0.70.15-py311-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]>=2021.11.1 from https://files.pythonhosted.org/packages/e3/bd/4c0a4619494188a9db5d77e2100ab7d544a42e76b2447869d8e124e981d8/fsspec-2023.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp from https://files.pythonhosted.org/packages/64/04/9ef622ccb6b340b3b53812e19f1658311614889452258eff91f6c9e1a1d9/aiohttp-3.8.5-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0.0,

/Users/bgrnaymane/Documents/GitHub/Projektrealisierung_Gruppe5/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  Using cached cupy-wheel-12.1.0.post1.tar.gz (2.9 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      /private/var/folders/6q/6zpwk60d759gx4wqgn31g22r0000gn/T/pip-install-i966mvqd/cupy-wheel_f3c7082e02c4440ebaa3af5201ac0e01/setup.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        import pkg_resources
      [cupy-wheel] CUDA detection unsupported on platform: darwin
      [cupy-wheel] ROCm detection unsupported on platform: darwin
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/6q/6zpwk60d759gx4wqgn31g22r0000gn/T/pip-install-i966mvqd/cupy-wheel_f3c7082e02c4440ebaa3af5201ac0e01/setup.py", line 285, in <module>
          main()
        Fi

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bgrnaymane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Einlesen des dfs

In [3]:
df = pd.read_csv("../data/Blogs_result/dataset.csv")
df = df.drop('Class', axis=1)

In [4]:
df_ref = pd.read_csv('/Users/bgrnaymane/Documents/GitHub/Projektrealisierung_Gruppe5/data/Blogs_result/blogsresult_refsummarys.csv')

In [5]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Erstellung TextRank-zusammenfassungsfunktion

In [6]:
def text_rank(text, compression_rate):
    sentences = sent_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    preprocessed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word.isalnum() and word not in stop_words]
        preprocessed_sentences.append(' '.join(words))
    
    vectorizer = CountVectorizer().fit_transform(preprocessed_sentences)
    similarity_matrix = cosine_similarity(vectorizer)
    
    damping_factor = 0.85  
    scores = np.ones(len(sentences))
    prev_scores = np.zeros(len(sentences))
    while np.sum(np.abs(scores - prev_scores)) > 0.001:
        prev_scores = np.copy(scores)
        for i in range(len(sentences)):
            scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[:, i] * scores) / np.sum(similarity_matrix[:, i])
    
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
    num_sentences_to_keep = max(1, int(compression_rate * len(sentences)))
    summary = [sentence for score, sentence in ranked_sentences[:num_sentences_to_keep]]
    
    return ' '.join(summary)

text = """
TextRank is a graph-based algorithm for automatic text summarization. It uses the PageRank algorithm to assign
importance scores to sentences in the text. The sentences with higher scores are considered more important and are
included in the summary. By adjusting the compression rate, you can control the length of the summary. TextRank
has been widely used in various natural language processing applications.
"""

 
summary = text_rank(text, 1)
print(summary)


The sentences with higher scores are considered more important and are
included in the summary. TextRank
has been widely used in various natural language processing applications. It uses the PageRank algorithm to assign
importance scores to sentences in the text. By adjusting the compression rate, you can control the length of the summary. 
TextRank is a graph-based algorithm for automatic text summarization.


### Vergleichen mit de Referenzzusammenfassungen

In [7]:
# Function for Tokenization, Remove stopwords, Lowercasing, Lemmatization, Remove punctuation
def preprocess_text(text):
    # Tokenization
    doc = nlp(text)

    # Remove stopwords, Lowercasing, Lemmatization, Remove punctuation
    preprocessed_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

    # Return the preprocessed tokens as text
    preprocessed_text = " ".join(preprocessed_tokens)
    return preprocessed_text

In [13]:
text = """
TextRank is a graph-based algorithm for automatic text summarization. It uses the PageRank algorithm to assign
importance scores to sentences in the text. The sentences with higher scores are considered more important and are
included in the summary. By adjusting the compression rate, you can control the length of the summary. TextRank
has been widely used in various natural language processing applications.
"""

sum= 'TextRank is a graph-based algorithm for automatic text summarization.'




0.14516129032258066


In [18]:
def evaluate_rouge_scores(df):
    rouge = Rouge()

    rouge_scores = {
        'all_classes': {
            'rouge-1': {
                'f': [],
                'p': [],
                'r': []
            }
        }
    }

    # Get unique classes
    classes = df['Class'].unique()

    for cls in classes:
        rouge_scores[cls] = {
            'rouge-1': {
                'f': [],
                'p': [],
                'r': []
            }
        }

    for _, row in df.iterrows():
        text = row['Text']
        reference_summary = row['Summary']  # Already preprocessed in the DataFrame
        cls = row['Class']

        # preprocessing
        preprocessed_text = str(text)
        preprocessed_reference_summary = str(reference_summary)

        #print(type(preprocessed_text))
        #print(type(preprocessed_reference_summary))

    
        # Generate summary with compression rate
        compression_rate = len(preprocessed_reference_summary.split())/len(preprocessed_text.split())
        summary = text_rank(preprocessed_text, compression_rate=compression_rate)

        

        # calculate rouge 1
        rouge_scores_all = rouge.get_scores(str(summary), str(reference_summary))[0]
        rouge_scores_cls = rouge_scores[cls]

        # Add rouge score values
        rouge_scores_all_cls = rouge_scores_cls['rouge-1']
        rouge_scores_all_cls['f'].append(rouge_scores_all['rouge-1']['f'])
        rouge_scores_all_cls['p'].append(rouge_scores_all['rouge-1']['p'])
        rouge_scores_all_cls['r'].append(rouge_scores_all['rouge-1']['r'])

        # Add rouge score values for all classes
        rouge_scores_all_all = rouge_scores['all_classes']['rouge-1']
        rouge_scores_all_all['f'].append(rouge_scores_all['rouge-1']['f'])
        rouge_scores_all_all['p'].append(rouge_scores_all['rouge-1']['p'])
        rouge_scores_all_all['r'].append(rouge_scores_all['rouge-1']['r'])

    return rouge_scores


In [19]:
rouge_scores = evaluate_rouge_scores(df_ref)

# calculate ROUGE-1-Scores for all classes
print("ROUGE-1 Scores für alle Klassen:")
print(rouge_scores['all_classes']['rouge-1'])

# calculate ROUGE-1-Scores for each class
for cls, scores in rouge_scores.items():
    if cls != 'all_classes':
        print(f"ROUGE-1 Scores für Klasse {cls}:")
        print(scores['rouge-1'])


/var/folders/6q/6zpwk60d759gx4wqgn31g22r0000gn/T/ipykernel_24024/1629045934.py:20: RuntimeWarning: invalid value encountered in scalar divide
  scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[:, i] * scores) / np.sum(similarity_matrix[:, i])


KeyboardInterrupt: 

# Implementierung A

In [ ]:
text_rank_summarizer = TextRankSummarizer()

compression_rates = [0.1, 0.3, 0.5, 0.7]  

approach_counts = {summarizer.__class__.__name__: 0 for summarizer in [text_rank_summarizer]}
compression_rate_counts = {rate: 0 for rate in compression_rates}

for index, row in df.iterrows():
    approach = random.choice([text_rank_summarizer])
    compression_rate = random.choice(compression_rates)
    
    approach_counts[approach.__class__.__name__] += 1
    compression_rate_counts[compression_rate] += 1

    tokenizer = Tokenizer("english")
    sentences = tokenizer.to_sentences(row['Text'])

    num_sentences = int(len(sentences) * compression_rate)

    summarizer = approach
    parser = PlaintextParser.from_string(row['Text'], tokenizer)
    summary = summarizer(parser.document, num_sentences)

    df.loc[index, 'Summary'] = ' '.join(str(sentence) for sentence in summary)


total_summaries = len(df)

approach_percentages = {approach: (count / total_summaries) * 100 for approach, count in approach_counts.items()}
compression_rate_percentages = {rate: (count / total_summaries) * 100 for rate, count in compression_rate_counts.items()}

for approach, percentage in approach_percentages.items():
    print(f"The approach {approach} was used for {percentage:.2f}% of the summaries.")

for rate, percentage in compression_rate_percentages.items():
    print(f"The compression rate {rate} was used for {percentage:.2f}% of the summaries.")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
import pandas as pd

folder_path = '../data/Blogs_result/'

file_name = 'blogsresult_just_textrank.csv'

file_path = folder_path + '/' + file_name

df.to_csv(file_path, index=False)

print(f"DataFrame saved as CSV file at: {file_path}")

## Evaluierung

In [ ]:
df1 = pd.read_csv("../data/Blogs_result/blogsresult_just_textrank.csv")
df2 = pd.read_csv("../data/Blogs_result/blogsresult_refsummarys.csv")


In [ ]:
from rouge import Rouge

rouge = Rouge()

rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []

for idx, row1 in df1.iterrows():
    summary1 = str(row1['Summary'])
    
    row2 = df2.loc[idx]
    summary2 = str(row2['Summary'])
    
    scores = rouge.get_scores(summary1, summary2)[0]
    
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)

min_rouge_1 = min(rouge_1_scores)
mean_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores)
max_rouge_1 = max(rouge_1_scores)

min_rouge_2 = min(rouge_2_scores)
mean_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)
max_rouge_2 = max(rouge_2_scores)

min_rouge_l = min(rouge_l_scores)
mean_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
max_rouge_l = max(rouge_l_scores)

print("Rouge-1 scores:")
print(f"Min: {min_rouge_1}")
print(f"Mean: {mean_rouge_1}")
print(f"Max: {max_rouge_1}")

print("Rouge-2 scores:")
print(f"Min: {min_rouge_2}")
print(f"Mean: {mean_rouge_2}")
print(f"Max: {max_rouge_2}")

print("Rouge-L scores:")
print(f"Min: {min_rouge_l}")
print(f"Mean: {mean_rouge_l}")
print(f"Max: {max_rouge_l}")